# 2D Transient Stokes and Navier-Stokes Flow on Tank and Nozzle Geometry 

<b>Problem to Solve</b> 

<b>Goals</b>

<b>Inlet velocity conditions</b>
1. the <i>parabolic</i> inlet profile imposed is such that the velocity imposed is zero at the top and bottom of the nozzle, and thus matching with the no-slip boundary conditions on these patches;
2. the <i>amplitude</i> of the parabolic inlet velocity is linearly ramped in time (starting from zero ending in vmax), and thus the zero inlet guess (for both velocity and pressure) in compatible with the boundary conditions imposed. The ramping introduces a discontinuity at time $t=1$. This discontinuity is taken into account in the time-stepping procedure;

<b>Initial solution for time-stepping</b>
1. setting zero initial guess vs. Stokes solution as initial guess for time-stepping

<b>Questions</b>
1. run first tests using a coarse mesh (faster computations), a small inlet velocity and a large viscosity; 
2. what causes small initial time step? High inlet velocity (thus small CFL number?). Should the mesh be adapted to the flow? 

<b>Remarks</b>
1. function ferrite_limiter!(): in order to obtain correct higher order convergence for time-dependent Dirichlet conditions, one needs to ensure that all the internal buffers are also correctly set at all times. The limiter helps guarantees this (as the time integrators do also other stuff than just calling your right-hand side). See e.g. [rosenbrock_perform_step.jl](https://github.com/SciML/OrdinaryDiffEq.jl/blob/7a4a5fb6d95f78e973b3d7bb41861058d5b00881/lib/OrdinaryDiffEqRosenbrock/src/rosenbrock_perform_step.jl#L1386-L1388)
2. Rodas5P() failed: results in MethodError: no method matching Rodas5P(; autodiff::Bool, step_limiter!::typeof(ferrite_limiter!))

##  Import Packages

In [72]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 

More later.

## Section 2: Definition of Structs 

In [73]:
#..pass data to RHS of time integration 
struct RHSparams
    K::SparseMatrixCSC
    ch::ConstraintHandler
    dh::DofHandler
    cellvalues_v::CellValues
    u::Vector
end

struct FreeDofErrorNorm
    ch::ConstraintHandler
end

## Section 3: Read 2D Mesh from File and Perform Elementary Checks

In [74]:
grid  = togrid("tankAndNozzle.msh")
inlet_patch = getfacetset(grid, "inlet")
wall_patch  = getfacetset(grid, "wall");

Info    : Reading 'tankAndNozzle.msh'...
Info    : 17 entities
Info    : 14713 nodes
Info    : 29424 elements
Info    : Done reading 'tankAndNozzle.msh'


In [75]:
grid

Grid{2, Triangle, Float64} with 28698 Triangle cells and 14713 nodes

In [76]:
grid  = togrid("extrudeLine.msh")

Info    : Reading 'extrudeLine.msh'...
Info    : 9 entities
Info    : 171 nodes
Info    : 200 elements
Info    : Done reading 'extrudeLine.msh'


Grid{2, Quadrilateral, Float64} with 144 Quadrilateral cells and 171 nodes

## Section 4: Set-up 

In [61]:
function setup_mean_constraint(dh, fvp)
    assembler = Ferrite.COOAssembler()
    # All external boundaries
    set = union(
            getfacetset(dh.grid, "wall"),
            getfacetset(dh.grid, "inlet"),
    )
    # Allocate buffers
    range_p = dof_range(dh, :p)
    element_dofs = zeros(Int, ndofs_per_cell(dh))
    element_dofs_p = view(element_dofs, range_p)
    element_coords = zeros(Vec{2}, 3) # assuming 2D mesh with triangles only 
    Ce = zeros(1, length(range_p)) # Local constraint matrix (only 1 row)
    # Loop over all the boundaries
    for (ci, fi) in set
        Ce .= 0
        getcoordinates!(element_coords, dh.grid, ci)
        Ferrite.reinit!(fvp, element_coords, fi)
        celldofs!(element_dofs, dh, ci)
        for qp in 1:getnquadpoints(fvp)
            dΓ = getdetJdV(fvp, qp)
            for i in 1:getnbasefunctions(fvp)
                Ce[1, i] += shape_value(fvp, qp, i) * dΓ
            end
        end
        # Assemble to row 1
        assemble!(assembler, [1], element_dofs_p, Ce)
    end
    C, _ = finish_assemble(assembler)
    # Create an AffineConstraint from the C-matrix
    _, J, V = findnz(C)
    _, constrained_dof_idx = findmax(abs2, V)
    constrained_dof = J[constrained_dof_idx]
    V ./= V[constrained_dof_idx]
    mean_value_constraint = AffineConstraint(
        constrained_dof,
        Pair{Int,Float64}[J[i] => -V[i] for i in 1:length(J) if J[i] != constrained_dof],
        0.0,
    )

    return mean_value_constraint
end

setup_mean_constraint (generic function with 1 method)

In [62]:
dim = 2 

ip_v = Lagrange{RefTriangle, 2}()^dim
qr = QuadratureRule{RefTriangle}(4)
cellvalues_v = CellValues(qr, ip_v);

ip_p = Lagrange{RefTriangle, 1}()
cellvalues_p = CellValues(qr, ip_p);

ipg = Lagrange{RefTriangle,1}() # linear geometric interpolation
qr_facet = FacetQuadratureRule{RefTriangle}(2)
fvp = FacetValues(qr_facet, ip_p, ipg) # facet values for pressure  

dh = DofHandler(grid)
add!(dh, :v, ip_v)
add!(dh, :p, ip_p)
close!(dh);

ch = ConstraintHandler(dh);

#..Boundary conditions part (1/3): Dirichlet BC for the velocity at the top lid 
Rin = 0.02
vmax = 0.01 
inlet_patch = getfacetset(grid, "inlet")
vin(t) = min(t * vmax, vmax) #inflow velocity
parabolic_inflow_profile(x,t) = Vec(( vin(t)*(Rin + x[2]) * (Rin - x[2])/Rin^2,0.)) 
inflow_bc = Dirichlet(:v, inlet_patch, parabolic_inflow_profile, [1, 2])
add!(ch, inflow_bc);

#..Boundary conditions part (2/3): no slip boundary condition - impose velocity to be zero vector on the walls
wall_patch = getfacetset(grid, "wall")
wall_bc = Dirichlet(:v, wall_patch, (x, t) -> Vec((0.0, 0.0)), [1, 2])
add!(ch, wall_bc);

# Boundary conditions part (3/3): apply pressure constraint
mean_value_constraint = setup_mean_constraint(dh, fvp)
add!(ch, mean_value_constraint)

close!(ch)
update!(ch, 0.0);

In [63]:
# collect([ ri for ri in [-Rin:Rin/10:Rin] ])

## Section 5: Functions for Mass and Stiffness Matrix Assembly and Other Functions   

In [64]:
function assemble_mass_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    # Allocate a buffer for the local matrix and some helpers, together with the assembler.
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Mₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # It follows the assembly loop as explained in the basic tutorials.
    mass_assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Mₑ, 0)
        Ferrite.reinit!(cellvalues_v, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)
            # Remember that we assemble a vector mass term, hence the dot product.
            # There is only one time derivative on the left hand side, so only one mass block is non-zero.
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Mₑ[BlockIndex((v▄, v▄), (i, j))] += φᵢ ⋅ φⱼ * dΩ
                end
            end
        end
        assemble!(mass_assembler, celldofs(cell), Mₑ)
    end

    return M
end;

function assemble_stokes_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, ν, K::SparseMatrixCSC, dh::DofHandler)
    # Again, some buffers and helpers
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Kₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # Assembly loop
    stiffness_assembler = start_assemble(K)
    for cell in CellIterator(dh)
        # Don't forget to initialize everything
        fill!(Kₑ, 0)

        Ferrite.reinit!(cellvalues_v, cell)
        Ferrite.reinit!(cellvalues_p, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)

            for i in 1:n_basefuncs_v
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    ∇φⱼ = shape_gradient(cellvalues_v, q_point, j)
                    Kₑ[BlockIndex((v▄, v▄), (i, j))] -= ν * ∇φᵢ ⊡ ∇φⱼ * dΩ
                end
            end

            for j in 1:n_basefuncs_p
                ψ = shape_value(cellvalues_p, q_point, j)
                for i in 1:n_basefuncs_v
                    divφ = shape_divergence(cellvalues_v, q_point, i)
                    Kₑ[BlockIndex((v▄, p▄), (i, j))] += (divφ * ψ) * dΩ
                    Kₑ[BlockIndex((p▄, v▄), (j, i))] += (ψ * divφ) * dΩ
                end
            end
        end

        # Assemble `Kₑ` into the Stokes matrix `K`.
        assemble!(stiffness_assembler, celldofs(cell), Kₑ)
    end
    return K
end

function ferrite_limiter!(u, _, p, t)
    update!(p.ch, t)
    return apply!(u, p.ch)
end

function navierstokes_rhs_element!(dvₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            dvₑ[j] -= v ⋅ ∇v' ⋅ φⱼ * dΩ
        end
    end
    return
end

function stokes!(du, u_uc, p::RHSparams, t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    mul!(du, K, u) # du .= K * u

    return
end;

function navierstokes!(du, u_uc, p::RHSparams, t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    mul!(du, K, u) # du .= K * u

    # nonlinear contribution
    v_range = dof_range(dh, :v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    vₑ = zeros(n_basefuncs)
    duₑ = zeros(n_basefuncs)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]
        vₑ .= @views u[v_celldofs]
        fill!(duₑ, 0.0)
        navierstokes_rhs_element!(duₑ, vₑ, cellvalues_v)
        assemble!(du, v_celldofs, duₑ)
    end
    return
end;

function navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues_v, q_point, i)
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                Jₑ[j, i] -= (φᵢ ⋅ ∇v' + v ⋅ ∇φᵢ') ⋅ φⱼ * dΩ
            end
        end
    end
    return
end

function stokes_jac!(J, u_uc, p, t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    # Here we assume that J has exactly the same structure as K by construction
    nonzeros(J) .= nonzeros(K)

    return apply!(J, ch)
end

function navierstokes_jac!(J, u_uc, p, t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    # Here we assume that J has exactly the same structure as K by construction
    nonzeros(J) .= nonzeros(K)

    assembler = start_assemble(J; fillzero = false)

    # Assemble variation of the nonlinear term
    n_basefuncs = getnbasefunctions(cellvalues_v)
    Jₑ = zeros(n_basefuncs, n_basefuncs)
    vₑ = zeros(n_basefuncs)
    v_range = dof_range(dh, :v)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]

        vₑ .= @views u[v_celldofs]
        fill!(Jₑ, 0.0)
        navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
        assemble!(assembler, v_celldofs, Jₑ)
    end

    return apply!(J, ch)
end

navierstokes_jac! (generic function with 1 method)

## Section 6: Solving a Stokes Problem as Verification  

In [65]:
K = allocate_matrix(dh,ch);
viscosity = 1e3
K = assemble_stokes_matrix(cellvalues_v, cellvalues_p, viscosity, K, dh);

f = zeros(ndofs(dh))
update!(ch, 2.)
apply!(K, f, ch)
update!(ch, 0.)
u = K \ f;

VTKGridFile("init-tankNozzle-2d", dh) do vtk
    write_solution(vtk, dh, u)
    Ferrite.write_constraints(vtk, ch)
end

VTKGridFile for the closed file "init-tankNozzle-2d.vtu".

## Section 7: Perform Naive Time Integration 

In [68]:
T = 6.0
Δt₀ = 1e-6
Δt_save = 0.1

M = allocate_matrix(dh);
M = assemble_mass_matrix(cellvalues_v, cellvalues_p, M, dh);

K = allocate_matrix(dh);
viscosity = 1e3
K = assemble_stokes_matrix(cellvalues_v, cellvalues_p, viscosity, K, dh);

jac_sparsity = sparse(K);

apply!(M, ch)

# set initial guess 
uinit = zeros(ndofs(dh))

p = RHSparams(K, ch, dh, cellvalues_v, copy(uinit))

# rhs = ODEFunction(navierstokes!, mass_matrix = M; jac = navierstokes_jac!, jac_prototype = jac_sparsity)
rhs = ODEFunction(stokes!, mass_matrix = M; jac = stokes_jac!, jac_prototype = jac_sparsity)
problem = ODEProblem(rhs, uinit, (0.0, T), p);

(fe_norm::FreeDofErrorNorm)(u::Union{AbstractFloat, Complex}, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

problem = ODEProblem(navierstokes!, uinit, (0.0,0.1), p);
problem = ODEProblem(navierstokes!, uinit, (0.0,6), p);
# sol = solve(problem, ImplicitEuler())
# sol = solve(problem, ImplicitEuler(autodiff=false),abstol = 1.0e-5, reltol = 1.0e-5); 

In [56]:
# p1 = plot(sol.t)
# p2 = bar(sol.t,sol.t[2:end]-sol.t[1:end-1])
# plot(p1,p2,layout=(1,2))

## Section 8: Perform Advanced Time Integration and Post-Processing 

Even Stokes fails! 

1. timestepper methods include Euler (is explicit or forward Euler), ImplicitEuler
2. what is a sufficiently small time-step for an explicit time integration method? 

In [69]:
# timestepper = Rodas5P(autodiff = false, step_limiter! = ferrite_limiter!);
# timestepper = AutoTsit5(Rosenbrock23())(autodiff = false, step_limiter! = ferrite_limiter!);
# timestepper = AutoTsit5(autodiff = false, step_limiter! = ferrite_limiter!);
# timestepper = ImplicitEuler(autodiff=false)

#integrator = init(
#    problem, timestepper; initializealg = NoInit(), dt = Δt₀,
#    adaptive = true, abstol = 1.0e-4, reltol = 1.0e-5,
#    progress = true, progress_steps = 1,
#    verbose = true, internalnorm = FreeDofErrorNorm(ch), d_discontinuities = [1.0]
#);

timestepper = ImplicitEuler(autodiff=false)

integrator = init(
    problem, timestepper; dt = Δt₀,
    adaptive = true, abstol = 1.0e-4, reltol = 1.0e-5,
    progress = true, progress_steps = 1,
    verbose = true, internalnorm = FreeDofErrorNorm(ch), d_discontinuities = [1.0]
);

pvd = paraview_collection("tankNozzle-2d")
for (step, (u, t)) in enumerate(intervals(integrator))
    display(t)
    VTKGridFile("tankNozzle-2d-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

LoadError: InterruptException: